# Document Reader Experiments

In Question Answering (QA), queries are run over several documents to extract an answer to user questions, consisting of two main steps: (1) Document Retriever — retrieve the most useful documents that may contain the answer to a given question; (2) Document Reader — a machine reader carefully examines the retrieved documents and frame an answer.

In this Jupyter Notebook, we focused on the Document Reader experiments, motivated by the fact that using a good Reader (higher F1) produces a better and more concise response.

---
## Setup

Packages installation and setups.

### Run Configuration

Choose the dataset and the Document Reader algorithm.

In [ ]:
import enum

class Dataset(enum.Enum):
    '''Dataset options'''
    SQuAD = 1
    AdvQA = 2
    DuoRC = 3
    QASports = 4

class DocReader:
    '''Document Reader options'''
    BERT    = "deepset/bert-base-uncased-squad2"
    RoBERTa = "deepset/roberta-base-squad2"
    MiniLM  = "deepset/minilm-uncased-squad2"
    DistilBERT = "distilbert-base-uncased-distilled-squad"
    ELECTRA = "deepset/electra-base-squad2"
    SmallDistilBERT= "laurafcamargos/distilbert-qasports-basket-small"

class Sports:
    BASKETBALL = "basketball"
    FOOTBALL = "football"
    SOCCER = "soccer"
    ALL = ""


In [ ]:
# run configuration
NUM_K      = 1 # always = 1
DATASET    = Dataset.QASports
DOC_READER = DocReader.SmallDistilBERT
SPORT = Sports.BASKETBALL

### Package Installation

Install Haystack and HuggingFace packages.

In [ ]:
# Check if you have a GPU running
# The code runs in CPU as well
!nvidia-smi

Sat Jul 13 13:43:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# %%capture
# Install the Haystack
!pip install pip --quiet
!pip install farm-haystack[colab]
!pip install farm-haystack[metrics]
# !pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]
!pip install rapidfuzz
# Install Huggingface
!pip install datasets
!pip install transformers==4.20.1 --quiet
!pip install sentence-transformers==2.2.2 --quiet
!echo "Silent installation with success!"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.7/763.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 1

### Logging

We configure how logging messages should be displayed and which log level should be used before importing Haystack.

In [ ]:
import logging

# Setup Haystack logging format
logging.basicConfig(format="%(levelname)s - %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
!pip install mmh3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 3.0 MB/s eta 0:00:00


---
## Dataset

Importing and download the respective dataset.

### Abstract Dataset

In [ ]:
import pandas as pd
from abc import ABCMeta, abstractmethod

class AbstactDataset(metaclass = ABCMeta):
    '''Abstract dataset class'''

    def __init__(self):
        self.raw_dataset = self.download()
        self.df_dataset = self._transform_df()
        print(f"## {self.name} ##")
        print(self.raw_dataset)


    def _transform_df(self):
        '''Transform dataset in a pd.DataFrame'''
        return pd.DataFrame(self.raw_dataset)

    @property
    @abstractmethod
    def name(self):
        '''Dataset name'''
        pass

    @abstractmethod
    def download(self):
        '''Download the dataset'''
        pass

    @abstractmethod
    def get_documents(self):
        '''Get the unique documents to store in the Document Store'''
        pass


    @abstractmethod
    def get_validation(self):
        '''Get the validation set'''
        pass

### SQuaD Dataset

https://huggingface.co/datasets/squad

In [ ]:
import mmh3
from datasets import load_dataset
from haystack.schema import Label, Document, Answer
from haystack.schema import EvaluationResult, MultiLabel

class SQuadDataset(AbstactDataset):
    '''SQuaD Dataset'''
    name = "SQuaD Dataset"
    _columns = {
        "title": "title",
        "document": "context",
        "question": "question",
    }
    _metadata = {
        "dataset_id": "id"
    }

    def download(self):
        dataset = load_dataset("squad", split="validation")
        return dataset

    def get_documents(self):
        # Remove duplicated contents
        cc = self._columns
        dataset_name = f"{self.name}"
        df = self.df_dataset
        df = df.drop_duplicates(subset=[cc["title"], cc["document"]], keep="first")

        # Create Haystack Document objects
        list_docs = []
        skipped_count = 0  # contador de linhas puladas
        for _, row in df.iterrows():
            document_value = row[cc["document"]]
            if document_value is not None:
                document_value_bytes = str(document_value).encode('utf-8')  # Converte para bytes
                document_id = mmh3.hash128(document_value_bytes, signed=False)
                doc_metadata = {k: row[v] for k, v in self._metadata.items()}
                doc_metadata["title"] = row[cc["title"]]
                doc_metadata["dataset_name"] = dataset_name
                doc = Document(
                    id=document_id,
                    content_type="text",
                    content=document_value,
                    meta=doc_metadata
                )
                list_docs.append(doc)
            else:
                # Imprimir toda a linha do DataFrame correspondente ao documento
                print(f"Warning: 'document' is None for this row. Skipping document:")
                print(row)
                skipped_count += 1

        print(f"Total rows skipped due to 'document' being None: {skipped_count}")

        return list_docs

    def _get_answers(self, data):
        # Get question answer
        return data["answers"]["text"]

    def get_validation(self):
        # Get dataset info
        cc = self._columns
        df = self.df_dataset
        _self = self

        # Create Haystack labels
        eval_labels = []
        for _, row in df.iterrows():
            document_value = row[cc["document"]]
            if document_value is not None:
                document_id = mmh3.hash128(document_value, signed=False)
                doc_label = MultiLabel(labels=[
                    Label(
                        query=row[cc["question"]],
                        answer=Answer(answer=answer, type="extractive"),
                        document=Document(
                            id=document_id,
                            content_type="text",
                            content=document_value,
                        ),
                        is_correct_answer=True,
                        is_correct_document=True,
                        origin="gold-label",
                    )
                    for answer in _self._get_answers(row)
                ])
                eval_labels.append(doc_label)
            else:
                # Tratar caso em que o valor é None
                print("Warning: 'document' is None for this row in get_validation. Skipping...")

        return eval_labels


INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


### AdversarialQA Dataset

https://huggingface.co/datasets/adversarial_qa

In [ ]:
class AdversarialQADataset(SQuadDataset):
    '''AdversarialQA Dataset'''
    name = "AdversarialQA Dataset"

    def download(self):
        dataset = load_dataset("adversarial_qa", "adversarialQA", split="validation")
        return dataset

### DuoRC Dataset

https://huggingface.co/datasets/duorc

In [ ]:
class DuoRCDataset(SQuadDataset):
    '''DuoRC  Dataset'''
    name = "DuoRC Dataset"
    _columns = {
        "title": "title",
        "document": "plot",
        "question": "question",
    }
    _metadata = {
        "dataset_id": "question_id"
    }

    def download(self):
        dataset = load_dataset("duorc", "SelfRC", split="validation")
        return dataset

    def _transform_df(self):
        '''Transform dataset in a pd.DataFrame'''
        df = pd.DataFrame(self.raw_dataset)
        # Get questions with answer
        return df[~df["no_answer"]]

    def _get_answers(self, data):
        # Get question answer
        print(data)
        return data["answers"]

###QASports Dataset
https://huggingface.co/datasets/PedroCJardim/QASports

In [ ]:
import ast

class QASportsDataset(SQuadDataset):
    '''QASports Dataset'''

    name = "QASports Dataset"

    _columns = {
        "title": "context_title",
        "document": "context",
        "question": "question",
    }
    _metadata = {
        "dataset_id": "id_qa"
    }

    def __init__(self, sport=None):
        self.sport = sport
        super().__init__()  # chama o construtor da classe base

    def download(self):
        if self.sport is not None:
            dataset = load_dataset("PedroCJardim/QASports", self.sport, split="validation")
            return dataset
        else:
            dataset = load_dataset("PedroCJardim/QASports", split="validation")
            return dataset
    def _get_answers(self, data):
        # Converte a string que representa um dicionário em um dicionário Python
        answer_dict = ast.literal_eval(data["answer"])

        # Retorna uma lista com o texto da resposta
        return [answer_dict["text"]]

### Download the dataset

Get the dataset and store the documents in the Document Store.

In [ ]:
def dataset_switch(choice):
    '''Get dataset class'''

    if choice == Dataset.SQuAD:
        return SQuadDataset()
    elif choice == Dataset.AdvQA:
        return AdversarialQADataset()
    elif choice == Dataset.DuoRC:
        return DuoRCDataset()
    elif choice == Dataset.QASports:
      return QASportsDataset(SPORT)
    else:
      return "Invalid dataset"

# Get the dataset
dataset = dataset_switch(DATASET)
docs = dataset.get_documents()

Generating train split:   0%|          | 0/185934 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23242 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23242 [00:00<?, ? examples/s]

## QASports Dataset ##
Dataset({
    features: ['id_qa', 'context_id', 'context', 'question', 'answer', 'context_title', 'context_categories', 'url'],
    num_rows: 23242
})
Total rows skipped due to 'document' being None: 0


---
## Document Reader

In this experiment, we explored three Transformer based models for extractive Question Answering using the [FARM framework](https://github.com/deepset-ai/FARM).
* [BERT paper](https://arxiv.org/abs/1810.04805), [implementation](https://huggingface.co/deepset/bert-base-uncased-squad2)
* [RoBERTa paper](https://arxiv.org/abs/1907.11692), [implementation](https://huggingface.co/deepset/roberta-base-squad2)
* [MiniLM paper](https://arxiv.org/abs/2002.10957), [implementation](https://huggingface.co/deepset/minilm-uncased-squad2)


In [ ]:
from haystack.nodes import FARMReader

# Get the reader
reader = FARMReader(DOC_READER, use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


Downloading:   0%|          | 0.00/477 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/minilm-uncased-squad2' (Bert)


Downloading:   0%|          | 0.00/127M [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/minilm-uncased-squad2' (Bert model) from model hub.


Downloading:   0%|          | 0.00/107 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
from haystack import Pipeline

# Build the pipeline
pipe = Pipeline()
pipe.add_node(component=reader, name='Reader', inputs=['Query'])

In [ ]:
# Testing the pipeline
from haystack.utils import print_answers

# Querying documents
question = "Who did the Raptors face in the first round of the 2015 Playoffs?"
prediction = pipe.run(query=question, documents=docs[0:10], params={"Reader": {"top_k": 3}})

# Print answer
print_answers(prediction)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 12.71 Batches/s]

'Query: Who did the Raptors face in the first round of the 2015 Playoffs?'
'Answers:'
[   <Answer {'answer': 'Washington Wizards', 'type': 'extractive', 'score': 0.8629469871520996, 'context': ' Award, becoming the first Raptor to do so. The Raptors faced the Washington Wizards in the first round of the 2015 Playoffs and lost four straight ga', 'offsets_in_document': [{'start': 232, 'end': 250}], 'offsets_in_context': [{'start': 66, 'end': 84}], 'document_ids': ['168073128841507576969410138308837846391'], 'meta': {'dataset_id': '49377324229723915348616814546446752473', 'title': 'Toronto Raptors | Basketball Wiki | Fandom', 'dataset_name': 'QASports Dataset'}}>,
    <Answer {'answer': 'West Virginia', 'type': 'extractive', 'score': 0.022460104897618294, 'context': '1991, 1993 South Florida 0 8 Syracuse 5 1981, 1988, 1992, 2005, 2006 West Virginia 1 2010 4 Villanova 1 1995 1 Virginia Tech 0 5,6 Notes: 1 Villanova ', 'offsets_in_document': [{'start': 109, 'end': 122}], 'offsets_in_context

---
## Evaluation

About the metrics, you can read the [evaluation](https://docs.haystack.deepset.ai/docs/evaluation) web page.

In [ ]:
%%time

# For testing purposes, running on the first 100 labels
# For real evaluation, you must remove the [0:100]
eval_labels = dataset.get_validation()

eval_docs = [[label.document for label in multi_label.labels] for multi_label in eval_labels]

eval_result = pipe.eval(labels=eval_labels, documents=eval_docs, params={"Reader": {"top_k": NUM_K}})

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 70.84 Batches/s]


CPU times: user 16 s, sys: 596 ms, total: 16.6 s
Wall time: 16.8 s


In [ ]:
pprint(eval_labels[0])

<MultiLabel: {'labels': [{'id': '207f53e0-70f3-4185-9ec2-de7a56b5fe15', 'query': 'In what year did he become a New York State Basketball Player?', 'document': {'id': '162155919273417084454839941680164668430', 'content': ' Early life Brand was born in Cortlandt Manor, New York. At the age of thirteen, he enrolled in Peekskill High School, where he was immediately added to the varsity basketball roster. He averaged 40 points and 20 rebounds per game, played AAU basketball with future NBA players Lamar Odom and Ron Artest, and by his senior year he was consistently ranked among the top high school basketball players in the country and was selected as New York State Mr.', 'content_type': 'text', 'meta': {}, 'id_hash_keys': ['content'], 'score': None, 'embedding': None}, 'is_correct_answer': True, 'is_correct_document': True, 'origin': 'gold-label', 'answer': {'answer': 'senior', 'type': 'extractive', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, '

In [ ]:
from pprint import pprint

# Get and print the metrics
metrics = eval_result.calculate_metrics()
pprint(metrics)

{'Reader': {'exact_match': 0.59,
            'f1': 0.6043181818181818,
            'num_examples_for_eval': 100.0}}


In [ ]:
# Print a detailed report
pipe.print_eval_report(eval_result)

================== Evaluation Report ==================
                   Pipeline Overview
                      Query
                        |
                        |
                      Reader
                        |
                        | exact_match:  0.43
                        | exact_match_top_1:  0.43
                        | f1: 0.464
                        | f1_top_1: 0.464
                        | num_examples_for_eval: 1e+02
                        | num_examples_for_eval_top_1: 1e+02
                        |
                      Output

                Wrong Reader Examples
Query: 
 	What is the name of the football team that is currently in the Championship League?
Gold Document Ids: 
 	
Metrics: 
 	exact_match: 0.0
 	f1: 0.0
Answers: 
 	answer: Farsley Celtic 
 	context: ford Park Avenue · Chester · Chorley · Curzon Ashton · Darlington · Farsley Celtic · Gateshead · Gloucester City · Guiseley · Hereford · Kettering Tow 
_________________________________